In [140]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import json
import pickle
import tqdm

from sklearn import linear_model
from sklearn.utils.testing import ignore_warnings

np.set_printoptions(precision=4, suppress=True)

from collections import Counter

import scipy.stats as sts

In [2]:
pd.options.display.max_columns = 150
pd.options.display.max_rows = 150

----

### Задание

**Background**: в спортивном “Что? Где? Когда?” соревнующиеся команды отвечают на одни и те же вопросы. После минуты обсуждения команды записывают и сдают свои ответы на карточках; побеждает тот, кто ответил на большее число вопросов. Турнир обычно состоит из нескольких десятков вопросов (обычно 36 или 45, иногда 60, больше редко). Часто бывают синхронные турниры, когда на одни и те же вопросы отвечают команды на сотнях игровых площадок по всему миру, т.е. в одном турнире могут играть сотни, а то и тысячи команд. Соответственно, нам нужно:
- построить рейтинг-лист, который способен нетривиально предсказывать результаты будущих турниров;
- при этом, поскольку ЧГК — это хобби, и контрактов тут никаких нет, игроки постоянно переходят из команды в команду, сильный игрок может на один турнир сесть поиграть за другую команду и т.д.; поэтому единицей рейтинг-листа должна быть не команда, а отдельный игрок;


### Структура решения

I часть, в которой генерятся фичи для задачи предсказания вероятностности ответить на вопрос

II часть, в которой я понял, что на тесте будут ровно те же игроки, что и на трейне, а значит можно переобучаться сколько душе угодно

III часть, где предсказываются результаты турниров 2020 года с известными составами, но неизвестными вопросами

IV часть, где вводится ЕМ-алгоритм для учета влияния скрытой переменной "сила команды"

V часть, с рейтинг-листом турниров по сложности вопросов.

----

### I часть, в которой генерятся фичи и решается задача предсказания вероятностности ответить на вопрос

Читаем данные и генерируем фичи: 
- характеристики игрока
  - сколько турниров cыграл игрок
  - средний процент верных ответов 
  - средний процент верных ответов по маскам из questionQty
  - relative_team_position - относительная позициия в турнире (относительная т.к. есть турниры на 20 команд и на 1000+ команд)

In [195]:
%%time
tournaments = pickle.load(open('./data/tournaments.pkl', 'rb'))
results = pickle.load(open('./data/results.pkl', 'rb'))
players = pickle.load(open('./data/players.pkl', 'rb'))

CPU times: user 21.6 s, sys: 2.93 s, total: 24.5 s
Wall time: 24.6 s


In [4]:
[ (k, v['name']) for k,v in tournaments.items() if v['dateStart'][:4] == '2019' ][:5]

[(4772, 'Синхрон северных стран. Зимний выпуск'),
 (4973, 'Балтийский Берег. 3 игра'),
 (4974, 'Балтийский Берег. 4 игра'),
 (4975, 'Балтийский Берег. 5 игра'),
 (4986, 'ОВСЧ. 6 этап')]

In [196]:
len([ (k, v['name']) for k,v in tournaments.items() if v['dateStart'][:4] == '2017' ]), \
len([ (k, v['name']) for k,v in tournaments.items() if v['dateStart'][:4] == '2018' ]), \
len([ (k, v['name']) for k,v in tournaments.items() if v['dateStart'][:4] == '2019' ]), \
len([ (k, v['name']) for k,v in tournaments.items() if v['dateStart'][:4] == '2020' ]),

(533, 622, 687, 418)

In [6]:
tournaments[5465]

{'dateEnd': '2019-05-19T18:00:00+03:00',
 'dateStart': '2019-05-18T12:00:00+03:00',
 'id': 5465,
 'name': 'Чемпионат России',
 'orgcommittee': [{'id': 31038,
   'name': 'Владимир',
   'patronymic': 'Владимирович',
   'surname': 'Сушков'},
  {'id': 26469,
   'name': 'Алексей',
   'patronymic': 'Владимирович',
   'surname': 'Рабин'},
  {'id': 25882,
   'name': 'Максим',
   'patronymic': 'Оскарович',
   'surname': 'Поташев'},
  {'id': 144,
   'name': 'Сергей',
   'patronymic': 'Леонидович',
   'surname': 'Абрамов'}],
 'questionQty': {'1': 15, '2': 15, '3': 15, '4': 15, '5': 15, '6': 15},
 'season': '/seasons/52',
 'synchData': None,
 'type': {'id': 2, 'name': 'Обычный'}}

In [7]:
results[5465][0]['mask'], [p['player']['id'] for p in results[5465][0]['teamMembers']]

('010111101111010110001000111101011010111011000111111111110011110100001010101111111111011110',
 [28751, 30152, 30270, 27822, 27403, 4270])

In [8]:
# for team in results[5465]:
#     summ = sum([int(s) for s in team['mask']])
#     print(team['mask'], [p['player']['id'] for p in team['teamMembers']], summ)

In [186]:
tournaments[6731]

KeyError: 6731

----

In [9]:
%%time
rows = []
for game_id in results:

    if tournaments[game_id]['dateStart'][:4] not in ['2019']:
        continue

    try:
        for team in results[game_id]:
            team_name = team['team']['name']
            position = team['position']
            team_id = team['team']['id']
            player_ids = sorted([p['player']['id'] for p in team['teamMembers']])
            team_mask = team['mask']
            tournaments_type = tournaments[game_id]['type']['name']
            tournaments_questionQty = tournaments[game_id]['questionQty']
            date_start = tournaments[game_id]['dateStart'][:10]
            
            rows.append((game_id, date_start, team_id, player_ids, team_mask, position, tournaments_type, tournaments_questionQty))
            
    except Exception as e:
        #print(game_id, e)
        continue

CPU times: user 2.6 s, sys: 55.2 ms, total: 2.65 s
Wall time: 2.7 s


In [10]:
df = pd.DataFrame(rows)

In [11]:
df.shape

(86638, 8)

In [12]:
df.head(3)

,0,1,2,3,4,5,6,7
0,4772,2019-01-05,45556,"[6212, 15456, 18036, 18332, 22799, 26089]",111111111011111110111111111100010010,1.0,Синхрон,"{'1': 12, '2': 12, '3': 12}"
1,4772,2019-01-05,1030,"[1584, 1585, 10998, 16206, 40840]",111111111011110100101111011001011010,5.5,Синхрон,"{'1': 12, '2': 12, '3': 12}"
2,4772,2019-01-05,4252,"[10187, 18168, 21060, 23513, 31332, 35850]",111111111011110101101111001011110000,5.5,Синхрон,"{'1': 12, '2': 12, '3': 12}"


In [13]:
df.columns = ['game_id', 'date_start', 'team_id', 'team', 'tmask', 'team_position', 'type', 'questionQty']

In [14]:
df.head(3)

,game_id,date_start,team_id,team,tmask,team_position,type,questionQty
0,4772,2019-01-05,45556,"[6212, 15456, 18036, 18332, 22799, 26089]",111111111011111110111111111100010010,1.0,Синхрон,"{'1': 12, '2': 12, '3': 12}"
1,4772,2019-01-05,1030,"[1584, 1585, 10998, 16206, 40840]",111111111011110100101111011001011010,5.5,Синхрон,"{'1': 12, '2': 12, '3': 12}"
2,4772,2019-01-05,4252,"[10187, 18168, 21060, 23513, 31332, 35850]",111111111011110101101111001011110000,5.5,Синхрон,"{'1': 12, '2': 12, '3': 12}"


#### Немного чистки: 

заменить '?' на 0, 'X' на ''

Команды, у которых длина маски меньше положенного - заполняем знаком 'N' ('no data')

то есть неполную маску 101101010111001 мы превращаем в 101101010111001NNNNNNNNNNNNNNNNNNNNN

In [ ]:
%%time
df = df[df.team.apply(len) != 0]
df = df[df.tmask.notnull()]

df.date_start = pd.to_datetime(df.date_start)

df.tmask = df.tmask.str.replace('?', '0')    # заменяем на ноль 
df.tmask = df.tmask.str.replace('X', '')     # 'X' - удаляем, т.к. у всех других команд на этом месте всегда тоже 'X'

df['tmask_sum'] = df.tmask.apply(lambda x: sum([int(s) for s in x ]))

# будем использовать как фичу
dd = {'Синхрон': '1', 'Асинхрон': '2', 'Обычный': '3', 'Общий зачёт': '3', 'Строго синхронный': '1'}
df.type = df.type.map(dd).fillna('1').astype(int)

# # 16 турниров, где у команд неодинаковая длина маски
# # турниры: [5462, 5553, 5554, 5703, 5705, 5760, 5864, 6026, 6085, 6090, 6249, 6254, 6255, 6265, 6307, 6308]
df['tmask_len'] = df.tmask.apply(len) 
zz = df.groupby('game_id').tmask_len.nunique().loc[lambda x: x != 1]
# можно удалить, но тогда теряем 10% датасета!
# df = df[~df.game_id.isin(zz.index)]
# поэтому мы будем заполнять символом N - No data
# то есть неполную маску 101101010111001 мы превращаем в 101101010111001NNNNNNNNNNNNNNNNNNNNN
df['tmask_max_len'] = df.game_id.map( df.groupby('game_id').tmask_len.max() )
condition = df.game_id.isin(zz.index)
df.loc[condition, 'tmask'] = df[condition].tmask.str.pad(width=999, side='right', fillchar='N')
df.loc[condition, 'tmask'] = df[condition].apply(lambda x: x.tmask[:x.tmask_max_len], axis=1)

In [ ]:
df.head()

перейдем от представления турнир-команда к представлению турнир-игрок

In [ ]:
%%time

# df2 = df[['game_id', 'date_start', 'team_id', 'team', 'tmask', 'team_position']].copy()
df2 = df.copy()

df2 = df2.explode('team')

df2 = df2.rename(columns={'team': 'player_id'})

df2['tmask_list'] = df2.tmask.apply(lambda x: [(question_id, y) for question_id, y in enumerate(x)])
# df2 = df2.drop(['tmask'], axis=1)

In [ ]:
df2.head()

In [ ]:
def tmask_1_rate(x):
    """ процент единичек """
    try: 
        mask_to_int = [int(i) for i in x if i != 'N']
        sum_ones = sum(mask_to_int)
        len_mask = len(mask_to_int)
        return sum_ones / len_mask
    except: 
        return np.NaN

In [ ]:
def tmask_N_rate(x):
    """ процент где вопрос это 'N' """
    mask_to_int = sum([1 for i in x if i == 'N'])
    return mask_to_int / len(x)

In [ ]:
%%time
df2['questions_hit_rate'] = df2.tmask.apply(tmask_1_rate)
df2['questions_N_rate'] = df2.tmask.apply(tmask_N_rate)

In [ ]:
df2['relative_team_position'] = df2.team_position / df2.game_id.map( df2.groupby('game_id').team_id.nunique() )

### questionQty

Вопросы разделены на несколько стадий, обычно 12 + 12 + 12 , но иногда бывает 15+15+15 или 36 + 36 + 36 + 36 + 36 + 36 и др.

Посчитаем по каждой стадии процент верных ответов т.к. есть гипотеза что первая стадия чуть проще чем последняя.

In [ ]:
df2.questionQty.astype(str).value_counts()[:20]

In [ ]:
condition = df2.questionQty.apply(len) > 0
df2.loc[condition, 'tmask_p1'] = df2.loc[condition, 'tmask'].apply(lambda x: x[: len(x) // 3])
df2.loc[condition, 'tmask_p2'] = df2.loc[condition, 'tmask'].apply(lambda x: x[len(x) // 3 : 2 * len(x) // 3])
df2.loc[condition, 'tmask_p3'] = df2.loc[condition, 'tmask'].apply(lambda x: x[2 * len(x) // 3 :])

condition = df2.questionQty.apply(len) % 2
df2.loc[condition, 'tmask_p1'] = df2.loc[condition, 'tmask'].apply(lambda x: x[: len(x) //2 ])
df2.loc[condition, 'tmask_p3'] = df2.loc[condition, 'tmask'].apply(lambda x: x[len(x) // 2 :])

condition = df2.questionQty.apply(len) % 4 == 0
df2.loc[condition, 'tmask_p1'] = df2.loc[condition, 'tmask'].apply(lambda x: x[: len(x) // 4])
df2.loc[condition, 'tmask_p2'] = df2.loc[condition, 'tmask'].apply(lambda x: x[len(x) // 4 : 2 * len(x) // 4])
df2.loc[condition, 'tmask_p2'] = df2.loc[condition, 'tmask'].apply(lambda x: x[len(x) // 4 : 3 * len(x) // 4])
df2.loc[condition, 'tmask_p3'] = df2.loc[condition, 'tmask'].apply(lambda x: x[3 * len(x) // 4 :])

condition = df2.questionQty.apply(len) % 3 == 0
df2.loc[condition, 'tmask_p1'] = df2.loc[condition, 'tmask'].apply(lambda x: x[: len(x) // 3])
df2.loc[condition, 'tmask_p2'] = df2.loc[condition, 'tmask'].apply(lambda x: x[len(x) // 3 : 2 * len(x) // 3])
df2.loc[condition, 'tmask_p3'] = df2.loc[condition, 'tmask'].apply(lambda x: x[2 * len(x) // 3 :])

condition = df2.questionQty.apply(len) % 5 == 0
df2.loc[condition, 'tmask_p1'] = df2.loc[condition, 'tmask'].apply(lambda x: x[: len(x) // 5 ])
df2.loc[condition, 'tmask_p1'] = df2.loc[condition, 'tmask'].apply(lambda x: x[len(x) // 5 : 2 * len(x) // 5 ])
df2.loc[condition, 'tmask_p2'] = df2.loc[condition, 'tmask'].apply(lambda x: x[2 * len(x) // 5 : 3 * len(x) // 5 ])
df2.loc[condition, 'tmask_p3'] = df2.loc[condition, 'tmask'].apply(lambda x: x[3 * len(x) // 5 : 4 * len(x) // 5 ])
df2.loc[condition, 'tmask_p3'] = df2.loc[condition, 'tmask'].apply(lambda x: x[4 * len(x) // 5  :])

In [ ]:
df2['tmask_p1_hit_rate'] = df2.tmask_p1.apply(tmask_1_rate)
df2['tmask_p2_hit_rate'] = df2.tmask_p2.apply(tmask_1_rate)
df2['tmask_p3_hit_rate'] = df2.tmask_p3.apply(tmask_1_rate)

In [ ]:
df2['tmask_len'] = df2.tmask.apply(len)

### Ключевой момент: джойн сам с собой + условие на дату, чтобы не заглядывать в будущее. То есть для каждой игры считаем статистики только по прошлым турнирам

In [ ]:
%%time
dff = pd.merge(df2[['date_start', 'player_id', 'game_id']], df2, on='player_id')
dff = dff[dff.date_start_y < dff.date_start_x]

In [ ]:
dff.head(5)

Считаем характеристики силы игрока 

- сколько турниров отыграл
- средний процент верных ответов
- какие занимал позиции в рейтинге: средняя, самая лучшая, самая худшая, дисперсия (aka постоянство)
- процент ответов по разным ступеням/стадиям вопросов (questionQty: part1 - part2 - part3)

In [ ]:
%%time
player_feats = \
    dff.groupby(['player_id', 'date_start_x', 'game_id_x'])\
       .agg(
            {'game_id_y': ['count']
            , 'team_position': ['max', 'min', 'mean', 'std']
            , 'relative_team_position': ['max', 'min', 'mean', 'std']
            , 'questions_hit_rate': ['min', 'max', 'mean', 'std']
            , 'questions_N_rate': ['min', 'max', 'mean', 'std']
            , 'tmask_p1_hit_rate': ['min', 'max', 'mean', 'std']
            , 'tmask_p2_hit_rate': ['min', 'max', 'mean', 'std']
            , 'tmask_p3_hit_rate': ['min', 'max', 'mean', 'std']
           })

In [ ]:
player_feats.columns = [i + '_' + j for i,j in player_feats.columns]

In [ ]:
player_feats = player_feats.reset_index()

In [ ]:
player_feats = player_feats.drop_duplicates(['player_id', 'date_start_x'])
player_feats = player_feats.drop('game_id_x', axis=1)

In [ ]:
player_feats.head(11)

In [ ]:
player_feats = player_feats.rename(columns={'date_start_x':'date_start'})

### Отлично, для каждого игрока получили признаки как он играл в прошлом. 

### Создадим матрицу игрок-вопрос и заджойним к ней фичи игрока

In [ ]:
%%time
usecols = ['game_id', 'date_start', 'team_id', 'player_id', 'tmask', 'tmask_list', 'team_position', 'type']
X = df2[usecols].explode('tmask_list')

X['question_id'] = X.tmask_list.apply(lambda x: x[0])
X['y'] = X.tmask_list.apply(lambda x: x[1])

X = X.drop(['tmask_list', 'tmask'], axis=1)

# удалить из обучения вопросы где не было ответа (No data)
X = X[X.y != 'N']
X.y = X.y.astype(int)

In [ ]:
X.head(15)

In [ ]:
X.shape

In [ ]:
%%time
X = pd.merge(X, player_feats, on=['player_id', 'date_start'], how='left')

In [ ]:
X.shape

In [ ]:
import gc; gc.collect()

### Теперь сделаем фичи сложности вопроса

Для вопросов посчитаем лишь один признак: априорную вероятность верного ответа

In [ ]:
question_feats = X.groupby(['game_id', 'question_id']).agg({'y': ['mean']}) 
question_feats.columns = ['question_hit_rate']

In [ ]:
question_feats.head(8)

In [ ]:
sns.distplot(question_feats.question_hit_rate)

In [ ]:
print('Априорная вероятность верного ответа в целом по всем вопросам и турнирам', question_feats.question_hit_rate.mean() )

In [45]:
%%time
X = pd.merge(question_feats, X, on=['game_id','question_id'], how='left')

CPU times: user 10.6 s, sys: 6.09 s, total: 16.7 s
Wall time: 16.8 s


In [46]:
X.shape

(20911025, 38)

In [47]:
X.head(3)

,game_id,question_id,question_hit_rate,date_start,team_id,player_id,team_position,type,y,game_id_y_count,team_position_max,team_position_min,team_position_mean,team_position_std,relative_team_position_max,relative_team_position_min,relative_team_position_mean,relative_team_position_std,questions_hit_rate_min,questions_hit_rate_max,questions_hit_rate_mean,questions_hit_rate_std,questions_N_rate_min,questions_N_rate_max,questions_N_rate_mean,questions_N_rate_std,tmask_p1_hit_rate_min,tmask_p1_hit_rate_max,tmask_p1_hit_rate_mean,tmask_p1_hit_rate_std,tmask_p2_hit_rate_min,tmask_p2_hit_rate_max,tmask_p2_hit_rate_mean,tmask_p2_hit_rate_std,tmask_p3_hit_rate_min,tmask_p3_hit_rate_max,tmask_p3_hit_rate_mean,tmask_p3_hit_rate_std
0,4772,0,0.892295,2019-01-05,45556,6212,1.0,1,1,1.0,2.0,2.0,2.0,NaN,0.026316,0.026316,0.026316,NaN,0.555556,0.555556,0.555556,NaN,0.0,0.0,0.0,NaN,0.666667,0.666667,0.666667,NaN,0.5,0.5,0.5,NaN,0.5,0.5,0.5,NaN
1,4772,0,0.892295,2019-01-05,45556,15456,1.0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4772,0,0.892295,2019-01-05,45556,18036,1.0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### fit model

In [48]:
drops = ['player_id', 'y', 'game_id', 'date_start', 'team_id' ,'player_id', 'team_position', 'question_id']
use_cols = [c for c in X.columns if c not in drops]
all_features = list(X.head()[use_cols].columns)

In [49]:
X.sample(5)

,game_id,question_id,question_hit_rate,date_start,team_id,player_id,team_position,type,y,game_id_y_count,team_position_max,team_position_min,team_position_mean,team_position_std,relative_team_position_max,relative_team_position_min,relative_team_position_mean,relative_team_position_std,questions_hit_rate_min,questions_hit_rate_max,questions_hit_rate_mean,questions_hit_rate_std,questions_N_rate_min,questions_N_rate_max,questions_N_rate_mean,questions_N_rate_std,tmask_p1_hit_rate_min,tmask_p1_hit_rate_max,tmask_p1_hit_rate_mean,tmask_p1_hit_rate_std,tmask_p2_hit_rate_min,tmask_p2_hit_rate_max,tmask_p2_hit_rate_mean,tmask_p2_hit_rate_std,tmask_p3_hit_rate_min,tmask_p3_hit_rate_max,tmask_p3_hit_rate_mean,tmask_p3_hit_rate_std
14178825,5822,22,0.637463,2019-12-12,69147,190704,475.0,1,1,48.0,1468.5,10.0,230.864583,280.285924,0.942623,0.147661,0.614678,0.209021,0.088889,0.777778,0.370065,0.152366,0.0,0.846154,0.054170,0.190046,0.000000,0.833333,0.389080,0.180839,0.000000,0.916667,0.373257,0.202149,0.000000,0.833333,0.353051,0.199722
9047409,5696,18,0.415877,2019-06-28,42453,82836,15.0,1,0,25.0,740.5,4.0,170.400000,189.404494,0.937151,0.046512,0.527127,0.265456,0.166667,0.769231,0.424110,0.177096,0.0,0.807692,0.058022,0.202224,0.083333,0.769231,0.451303,0.190007,0.083333,0.818182,0.424400,0.207429,0.083333,0.846154,0.395904,0.216933
758833,4986,34,0.533812,2019-02-15,66720,155806,421.0,1,0,5.0,786.5,27.0,336.000000,318.008058,0.850329,0.427035,0.683637,0.181428,0.305556,0.444444,0.355674,0.053448,0.0,0.000000,0.000000,0.000000,0.133333,0.416667,0.299394,0.110734,0.250000,0.416667,0.325000,0.061802,0.333333,0.583333,0.456667,0.116428
2764890,5156,14,0.378486,2019-05-10,71195,197329,113.0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16911179,5976,77,0.393939,2019-10-26,65571,210380,2.0,3,1,3.0,1760.0,679.5,1160.666667,549.856875,0.916667,0.338228,0.703602,0.317878,0.222222,0.346369,0.263604,0.071676,0.0,0.750000,0.305814,0.393682,0.222222,0.323944,0.265389,0.052578,0.250000,0.375000,0.312500,0.088388,0.166667,0.333333,0.250000,0.117851


Убираем из обучения игроков, которые ранее не сыграли ни одной игры т.к. там скорее всего получится что-то тривиальное, типа априорная вероятность правильного ответа для человека без рейтинга

In [50]:
print(X.shape)
X.game_id_y_count = X.game_id_y_count.fillna(0)
X = X[X.game_id_y_count != 0]
print(X.shape)

(20911025, 38)
(17346457, 38)


In [51]:
fillna_dict = {}

для фич std заполним пропуски нулем

In [52]:
std_cols = [i for i in X.columns if i.endswith('std')]

for col in std_cols:
    X[col] = X[col].fillna(0)
    fillna_dict[col] = 0

для остальных колонок заполним пропуски средним значением

In [53]:
for col in all_features:
    mean = X[col].mean()
    X[col] = X[col].fillna(mean)
    fillna_dict[col] = mean

In [54]:
%%time
X.to_hdf('./data/X_logreg.h5', key='qwerty')

CPU times: user 3.17 s, sys: 6.44 s, total: 9.61 s
Wall time: 27.8 s


----

In [55]:
%%time
X = pd.read_hdf('./data/X_logreg.h5')

CPU times: user 8.17 s, sys: 4.52 s, total: 12.7 s
Wall time: 12.7 s


In [56]:
drops = ['player_id', 'y', 'game_id', 'date_start', 'team_id' ,'player_id', 'team_position', 'question_id']
use_cols = [c for c in X.columns if c not in drops]
all_features = list(X.head()[use_cols].columns)

In [57]:
from sklearn.linear_model import LogisticRegression

In [58]:
X.head(10)[all_features]

,question_hit_rate,type,game_id_y_count,team_position_max,team_position_min,team_position_mean,team_position_std,relative_team_position_max,relative_team_position_min,relative_team_position_mean,relative_team_position_std,questions_hit_rate_min,questions_hit_rate_max,questions_hit_rate_mean,questions_hit_rate_std,questions_N_rate_min,questions_N_rate_max,questions_N_rate_mean,questions_N_rate_std,tmask_p1_hit_rate_min,tmask_p1_hit_rate_max,tmask_p1_hit_rate_mean,tmask_p1_hit_rate_std,tmask_p2_hit_rate_min,tmask_p2_hit_rate_max,tmask_p2_hit_rate_mean,tmask_p2_hit_rate_std,tmask_p3_hit_rate_min,tmask_p3_hit_rate_max,tmask_p3_hit_rate_mean,tmask_p3_hit_rate_std
0,0.892295,1,1.0,2.0,2.0,2.00,0.00000,0.026316,0.026316,0.026316,0.000000,0.555556,0.555556,0.555556,0.000000,0.0,0.0,0.0,0.0,0.666667,0.666667,0.666667,0.000000,0.500000,0.500000,0.500000,0.000000,0.500000,0.500000,0.500000,0.000000
29,0.892295,1,1.0,1.0,1.0,1.00,0.00000,0.013158,0.013158,0.013158,0.000000,0.611111,0.611111,0.611111,0.000000,0.0,0.0,0.0,0.0,0.750000,0.750000,0.750000,0.000000,0.583333,0.583333,0.583333,0.000000,0.500000,0.500000,0.500000,0.000000
34,0.892295,1,1.0,24.0,24.0,24.00,0.00000,0.134831,0.134831,0.134831,0.000000,0.638889,0.638889,0.638889,0.000000,0.0,0.0,0.0,0.0,0.416667,0.416667,0.416667,0.000000,0.666667,0.666667,0.666667,0.000000,0.833333,0.833333,0.833333,0.000000
35,0.892295,1,2.0,75.0,20.5,47.75,38.53732,0.421348,0.269737,0.345543,0.107206,0.333333,0.500000,0.416667,0.117851,0.0,0.0,0.0,0.0,0.166667,0.416667,0.291667,0.176777,0.250000,0.416667,0.333333,0.117851,0.583333,0.666667,0.625000,0.058926
37,0.892295,1,1.0,24.0,24.0,24.00,0.00000,0.134831,0.134831,0.134831,0.000000,0.638889,0.638889,0.638889,0.000000,0.0,0.0,0.0,0.0,0.416667,0.416667,0.416667,0.000000,0.666667,0.666667,0.666667,0.000000,0.833333,0.833333,0.833333,0.000000
38,0.892295,1,1.0,24.0,24.0,24.00,0.00000,0.134831,0.134831,0.134831,0.000000,0.638889,0.638889,0.638889,0.000000,0.0,0.0,0.0,0.0,0.416667,0.416667,0.416667,0.000000,0.666667,0.666667,0.666667,0.000000,0.833333,0.833333,0.833333,0.000000
39,0.892295,1,1.0,24.0,24.0,24.00,0.00000,0.134831,0.134831,0.134831,0.000000,0.638889,0.638889,0.638889,0.000000,0.0,0.0,0.0,0.0,0.416667,0.416667,0.416667,0.000000,0.666667,0.666667,0.666667,0.000000,0.833333,0.833333,0.833333,0.000000
41,0.892295,1,1.0,5.5,5.5,5.50,0.00000,0.072368,0.072368,0.072368,0.000000,0.472222,0.472222,0.472222,0.000000,0.0,0.0,0.0,0.0,0.416667,0.416667,0.416667,0.000000,0.416667,0.416667,0.416667,0.000000,0.583333,0.583333,0.583333,0.000000
42,0.892295,1,2.0,5.5,5.5,5.50,0.00000,0.072368,0.030899,0.051634,0.029323,0.472222,0.722222,0.597222,0.176777,0.0,0.0,0.0,0.0,0.416667,0.750000,0.583333,0.235702,0.416667,0.750000,0.583333,0.235702,0.583333,0.666667,0.625000,0.058926
43,0.892295,1,1.0,5.5,5.5,5.50,0.00000,0.072368,0.072368,0.072368,0.000000,0.472222,0.472222,0.472222,0.000000,0.0,0.0,0.0,0.0,0.416667,0.416667,0.416667,0.000000,0.416667,0.416667,0.416667,0.000000,0.583333,0.583333,0.583333,0.000000


In [59]:
X.y.values

array([1, 1, 1, ..., 0, 0, 0])

In [60]:
import datetime
print(datetime.datetime.now())

2021-04-16 23:57:39.490948


In [61]:
%%time
lr = LogisticRegression(max_iter=1000, n_jobs=1)
lr.fit(X[all_features].values, X.y.values)

CPU times: user 2h 27min 19s, sys: 14min 18s, total: 2h 41min 38s
Wall time: 2h 35min


In [62]:
list(zip(lr.coef_.tolist()[0], all_features))

[(5.240661007879158, 'question_hit_rate'),
 (0.06476280527707987, 'type'),
 (0.0010290978770671432, 'game_id_y_count'),
 (-6.587867290069657e-05, 'team_position_max'),
 (0.0008968462158393122, 'team_position_min'),
 (-0.0009790850899672277, 'team_position_mean'),
 (0.00042593807115461226, 'team_position_std'),
 (0.15980561806018764, 'relative_team_position_max'),
 (-0.5758715990329295, 'relative_team_position_min'),
 (-1.6939708922900651, 'relative_team_position_mean'),
 (0.03490467756754931, 'relative_team_position_std'),
 (-0.022270717085329253, 'questions_hit_rate_min'),
 (-0.05369105944082633, 'questions_hit_rate_max'),
 (0.28467161497980026, 'questions_hit_rate_mean'),
 (-0.040419909843831585, 'questions_hit_rate_std'),
 (-0.027226749777632172, 'questions_N_rate_min'),
 (0.06350875824537207, 'questions_N_rate_max'),
 (0.010402387432406587, 'questions_N_rate_mean'),
 (0.049731423391614106, 'questions_N_rate_std'),
 (0.026139221271922686, 'tmask_p1_hit_rate_min'),
 (0.00500740542776

### II часть, в которой я понял, что на тесте будут ровно те же игроки, что и на трейне, а значит можно переобучаться сколько душе угодно

Обучили логистическую регрессию, теперь делаем predict_proba сам на себя (на игроков из трейна) и строим рейтинг игрока как мат.ожидание набранных баллов на "среднем" турнире со "средними" по сложности вопросами

На самом деле нужен даже не целый турнир, а просто один "средний" вопрос. 

Мы будем предсказывать вероятность дать правильный ответ - это и будет рейтинг игрока.

In [63]:
# фичи берем по последней игре данного игрока т.к. там информация о всех прошлых играх
player_feats['max_date_start'] = player_feats.player_id.map(player_feats.groupby('player_id').date_start.max())
XX = player_feats[player_feats.date_start == player_feats.max_date_start]

In [64]:
# вместо фич по вопросам подставляем "средние" значения
XX['question_hit_rate'] = 0.4253

In [65]:
# тип соревнования 
XX['type'] = 2   

In [66]:
# аналогично заполняем пропуски
# заполняем пропуски теми же средними, что были на этапе обучения модели
for col in all_features:
    fill_value = fillna_dict[col]           
    XX[col] = XX[col].fillna(fill_value)

In [67]:
rating = XX[['player_id']] # .rename(columns={'player_id': 'ID'})

In [68]:
rating['rating'] = lr.predict_proba(XX[all_features])[:, 1]

In [69]:
rating['player'] = rating.player_id.map( {k: v.get('name', '') + ' '+ v.get('surname', '') 
                                          for k,v in players.items()} )

In [70]:
rating = rating.sort_values('rating', ascending=False).reset_index(drop=True)

In [88]:
!mkdir -p output
rating.to_csv('./output/rating.csv', index=False)

In [71]:
rating.head(20)

,player_id,rating,player
0,28751,0.729801,Иван Семушин
1,30152,0.725588,Артём Сорожкин
2,4270,0.724764,Александра Брутер
3,33032,0.722046,Татьяна Фёдорова
4,30270,0.719878,Сергей Спешков
5,27822,0.719542,Михаил Савченков
6,7008,0.719095,Алексей Гилёв
7,18332,0.718219,Александр Либер
8,110920,0.715694,Алексей Дворянчиков
9,27499,0.715305,Наталия Рыжанова


In [72]:
# топ 100 по версии https://rating.chgk.info/players.php на начало апреля 2021
top100_real = ['Артём Сорожкин','Михаил Савченков','Иван Семушин','Сергей Спешков','Максим Руссо',
               'Александра Брутер','Александр Либер','Михаил Левандовский','Ким Галачян','Сергей Николенко','Тимур Кафиатуллин',
               'Юрий Выменец','Антон Чернин','Наиль Фарукшин','Сергей Коновалов','Андрей Островский','Игорь Тюнькин',
               'Татьяна Фёдорова','Екатерина Лобкова','Андрей Волыхов','Кирилл Чернышёв','Дмитрий Ожигов',
               'Наталья Кудряшова','Дмитрий Петров','Руслан Хаиткулов','Елизавета Овдеенко','Дмитрий Карякин',
               'Сергей Терентьев','Егор Дружинин','Алексей Гилёв','Максим Поташев','Дмитрий Великов',
               'Вадим Яковлев','Евгений Коватенков','Николай Некрылов','Инна Семёнова','Александр Салита',
               'Станислав Мереминский','Серафим Шибанов','Михаил Малкин','Алексей Дворянчиков','Ольга Сарницкая',
               'Владимир Степанов','Александр Карчевский','Екатерина Новикова','Ринат Сибаев','Сергей Иванченко',
               'Наталия Рыжанова','Александр Мартынов','Михаил Новосёлов','Александр Марков',
               'Иван Ефремов','Рузель Халиуллин','Алексей Шередега','Эльдар Эльман','Никита Поверинов',
               'Ася Самойлова','Мария Подрядчикова','Александр Печеный','Юлия Архангельская','Глеб Николаев',
               'Александр Фингеров','Владислав Декалов','Павел Ершов','Ирина Прокофьева','Андрей Белов',
               'Анвар Мухаметкалиев','Иделия Айзятулова','Дмитрий Тарарыков','Наталья Комар','Вячеслав Колосов',
               'Юлия Дидбаридзе','Владислав Харитонов','Игорь Мокин','Евгений Перпер','Леонид Михлин',
               'Николай Крапиль','Тимур Боков','Владимир Сушков','Николай Порцель','Ирина Проскурина',
               'Карина Файзуллина','Анастасия Шестакова','Дмитрий Литвинов','Игорь Биткин','Андрей Цепаев',
               'Екатерина Шевцова','Станислав Мальчёнков','Егор Кузьменко','Вадим Раскумандрин','Эльмира Гулуева',
               'Валерия Кудрявцева','Сергей Евсеев','Алексей Чебыкин','Антон Бочкарёв','Денис Галиакберов',
               'Мария Кленницкая','Анастасия Рубашкина','Алексей Шуб','Антон Пинчук', 
              ]

top100_real_enumerated = {j:i for i,j in enumerate(top100_real, start=1)}

In [81]:
top100 = rating.head(100)

In [82]:
top100['is_in_real_top100'] = top100.player.isin(top100_real).astype(int)
top100['real_rating'] = top100.player.map(top100_real_enumerated)

In [83]:
top100.head(20)

,player_id,rating,player,is_in_real_top100,real_rating
0,28751,0.729801,Иван Семушин,1,3.0
1,30152,0.725588,Артём Сорожкин,1,1.0
2,4270,0.724764,Александра Брутер,1,6.0
3,33032,0.722046,Татьяна Фёдорова,1,18.0
4,30270,0.719878,Сергей Спешков,1,4.0
5,27822,0.719542,Михаил Савченков,1,2.0
6,7008,0.719095,Алексей Гилёв,1,30.0
7,18332,0.718219,Александр Либер,1,7.0
8,110920,0.715694,Алексей Дворянчиков,1,41.0
9,27499,0.715305,Наталия Рыжанова,1,48.0


In [84]:
top100.is_in_real_top100.sum()

59

### 59 игроков из топ-100 официального рейтинга ЧГК попали в топ-100 по модели.

----

### III часть, где предсказываются результаты турниров 2020 года с известными составами, но неизвестными вопросами

_Качество рейтинг-системы оценивается качеством предсказаний результатов турниров_

_Поэтому предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;_

Мат.ожадание набранных командой баллов - это просто сумма вероятностей ответить на вопросы. 

Но так как заранее мы не знаем сложность вопросов, можно просто оценить вероятность ответить на один "средний" вопрос. 

Рейтинг команды считаем как вероятность, что команда ответит на этот "средний" вопрос. Она равняется тому, что хотя бы один член команды ответит правильно. (сделаем упрощающее предположение, что если хотя бы один член команды дает верный ответ, то и вся команда дает верный ответ)

$$P_{team}(y=1)= 1 - \prod	P_i(y=0)$$

где P_i(y=0) - индивидуальная вероятность, что i-ый член команды не дал правильного ответа на вопрос

In [85]:
rating.head(3)

,player_id,rating,player
0,28751,0.729801,Иван Семушин
1,30152,0.725588,Артём Сорожкин
2,4270,0.724764,Александра Брутер


In [245]:
rating_dict = rating[['player_id', 'rating']].set_index('player_id').T.to_dict('records')[0]


# {28751: [0.7298011491856893],
#  30152: [0.7255879465278992],
#  4270: [0.7247644842290046],
#  33032: [0.7220455169364687],
#  30270: [0.7198782130663005],
#  27822: [0.7195424698126476],
#  7008: [0.7190946342765561],
#  18332: [0.7182192841874325],
#  110920: [0.7156937105685747],
#  27499: [0.7153050188058772],
#  12313: [0.7149150113903385],
#  27403: [0.7130734824832596],
#  14786: [0.7109388032243028],
#  22799: [0.7102399948436137],
# ...}

In [246]:
rating_dict[28751], rating_dict[22799]

(0.7298011491856893, 0.7102399948436137)

In [413]:
%%time
rows = []
known_players = set(rating.player_id)

for game_id in results:

    if tournaments[game_id]['dateStart'][:4] not in ['2020']:
        continue

    try:
        for team in results[game_id]:
            team_name = team['team']['name']
            position = team['position']
            team_id = team['team']['id']
            player_ids = sorted([p['player']['id'] for p in team['teamMembers']])

            product = 1
            for player_id in player_ids:
                if player_id in known_players:
                    product *= (1 - rating_dict[player_id])
            team_rating = 1 - product            
            
            rows.append((game_id, team_name, position, team_id, player_ids, team_rating))
            
    except Exception as e:
        print(game_id, e)
        continue

5670 'position'
5714 'position'
5816 'position'
5922 'position'
6243 'position'
6309 'position'
6316 'position'
6336 'position'
6348 'position'
6350 'position'
6363 'position'
6364 'position'
6365 'position'
6392 'position'
6400 'position'
6415 'position'
6453 'position'
6469 'position'
6470 'position'
6471 'position'
CPU times: user 180 ms, sys: 29.4 ms, total: 209 ms
Wall time: 214 ms


In [414]:
df = pd.DataFrame(rows)

In [415]:
df.head(3)

,0,1,2,3,4,5
0,4957,Борский корабел,1.0,49804,"[4270, 27403, 27822, 28751, 30152, 30270]",0.999540
1,4957,Первая сборная,2.0,4109,"[25177, 33792, 34936, 40877, 107161, 113703]",0.998611
2,4957,Сова при свете дня,3.0,3875,"[13857, 19632, 21346, 33620, 37836, 46339]",0.997776


In [416]:
df.columns = ['game_id', 'team_name', 'position', 'team_id', 'player_ids', 'team_rating']

In [417]:
df.head(3)

,game_id,team_name,position,team_id,player_ids,team_rating
0,4957,Борский корабел,1.0,49804,"[4270, 27403, 27822, 28751, 30152, 30270]",0.999540
1,4957,Первая сборная,2.0,4109,"[25177, 33792, 34936, 40877, 107161, 113703]",0.998611
2,4957,Сова при свете дня,3.0,3875,"[13857, 19632, 21346, 33620, 37836, 46339]",0.997776


In [418]:
groud_truth_list = df.groupby('game_id').position.apply(list)
model_rating_list = df.groupby('game_id').team_rating.apply(list)

In [419]:
spearmen_corr = []
kendall_corr = []

for actual, model in zip(groud_truth_list, model_rating_list):
    
    spearman = sts.spearmanr(actual, np.argsort(-np.array(model))).correlation
    kendall = sts.kendalltau(actual, np.argsort(-np.array(model))).correlation

    if np.isnan(spearman):
        continue
    
    spearmen_corr.append(spearman)
    kendall_corr.append(kendall)
    
spearmen_score = np.mean(spearmen_corr)
kendall_score = np.mean(kendall_corr)

In [420]:
spearmen_score, kendall_score

(0.7159610793541057, 0.5597787726548374)

----

### IV часть, где вводится ЕМ-алгоритм для учета влияния скрытой переменной "сила команды"


_Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:_

_предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;_


- Е-шаг: по вероятностям (рейтингам) игроков считаем рейтинг команды - вероятность команды верно ответить на вопрос = 1 - П P(y = 0)
- М-шаг: с помощью вероятности команды пересчитываем вероятность игрока ответить на вопрос (то есть теперь уже при условии команды)

----

### V часть, с рейтинг-листом турниров по сложности вопросов.